In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import cvxpy as cp

In [2]:
stocks = pd.read_csv('Data_clean/6_Portfolios_ME_Prior_12_2_returns.csv')
bonds = pd.read_csv('Data_clean/bond_returns.csv')
riskfree = pd.read_csv('Data_clean/FF_cleaned.csv')

stocks = stocks[['Date', 'Market Return']]
stocks = stocks.rename(columns={'Market Return': 'Stock Return'})
stocks.set_index('Date', inplace=True)
stocks['Stock Return'] = stocks['Stock Return'] / 100

bonds = bonds[['Date', '10YrReturns']]
bonds = bonds.rename(columns={'10YrReturns': 'Bond Return'})
bonds.set_index('Date', inplace=True)

riskfree = riskfree[['Date', 'RF']]
riskfree.set_index('Date', inplace=True)
riskfree['RF'] = riskfree['RF'] / 100

data_all = pd.concat([stocks, bonds, riskfree], axis=1)
data = data_all[data_all.index >= '1990-01-31']
data_minus_rf = data[['Stock Return', 'Bond Return']]

In [3]:
target_return = 0.0075
target_return_60_40 = 0.007315
mu = data_minus_rf.mean()
sigma = data_minus_rf.cov()
mu_3 = data.mean()
sigma_3 = data.cov()

60/40 strategy:

In [4]:
weights_60_40 = np.array([0.6, 0.4])
exp_return_60_40 = np.dot(weights_60_40, mu)
risk_60_40 = np.sqrt(np.dot(weights_60_40.T, np.dot(sigma, weights_60_40)))
print('60/40:', 'Stocks', weights_60_40[0], 'Bonds', weights_60_40[1])
print('Expected return:', exp_return_60_40.round(6))
print('std:', risk_60_40.round(6))

60/40: Stocks 0.6 Bonds 0.4
Expected return: 0.007315
std: 0.027235


Is there a portfolio that delivers expected monthly return 0.007315 like the 60/40 but has lower risk?

Answer: No, just run markowitz unleveraged - closed solution 2 assets with the 0.007315 as new return target.

### Markowitz (unleveraged - closed solution): 2 - assets 

In [5]:
sigma_inv = np.linalg.inv(sigma)
ones = np.ones(len(mu))
a = mu.transpose() @ sigma_inv @ mu
b = mu.transpose() @ sigma_inv @ ones
c = ones.transpose() @ sigma_inv @ ones
A = np.array([[a, b], [b, c]])
A_inv = np.linalg.inv(A)
e = np.array([target_return, 1])

In [8]:
weights_ma_u = sigma_inv @ np.array([mu,ones]).T @ A_inv @ np.array([target_return, 1])
exp_return_ma_u = weights_ma_u @ mu
risk_ma_u = np.sqrt(weights_ma_u @ sigma @ weights_ma_u)
print('Markowitz (unleveraged): ', 'Stocks', weights_ma_u[0].round(5),'Bonds', weights_ma_u[1].round(5))
print('Expected return:', (exp_return_ma_u).round(4))
print('std:', (risk_ma_u).round(6))

Markowitz (unleveraged):  Stocks 0.6371 Bonds 0.3629
Expected return: 0.0075
std: 0.028624


### Markowitz (unleveraged - scipy optimization):

In [9]:
def objective(weights, sigma):
    return 0.5 * weights @ sigma @ weights

constraints = (
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}, 
    {'type': 'eq', 'fun': lambda weights: np.dot(weights, mu) - target_return}  
)
bounds = tuple((0, 1) for asset in range(len(mu)))

In [12]:
initial_weights = np.array([0.5,0.5])
result = minimize(objective, initial_weights, args=(sigma), method='SLSQP', bounds=bounds, constraints=constraints)
print("Markowitz (unleveraged):", result.x.round(5))
print("Expected return:", np.dot(result.x, mu).round(4))
print("std:", np.sqrt(objective(result.x, sigma)).round(4))

Markowitz (unleveraged): [0.6371 0.3629]
Expected return: 0.0075
std: 0.0202


### Markowitz (closed solution): 3 assets

In [13]:
sigma_inv_3 = np.linalg.inv(sigma_3)
ones_3 = np.ones(len(mu_3))
a_3 = mu_3.transpose() @ sigma_inv_3 @ mu_3
b_3 = mu_3.transpose() @ sigma_inv_3 @ ones_3
c_3 = ones_3.transpose() @ sigma_inv_3 @ ones_3
A_3 = np.array([[a_3, b_3], [b_3, c_3]])
A_inv_3 = np.linalg.inv(A_3)
e_3 = np.array([target_return, 1])

In [14]:
weights_ma_u_3 = sigma_inv_3 @ np.array([mu_3,ones_3]).T @ A_inv_3 @ np.array([target_return, 1])
exp_return_ma_u_3 = weights_ma_u_3 @ mu_3
risk_ma_u_3 = np.sqrt(weights_ma_u_3 @ sigma_3 @ weights_ma_u_3)
print('Markowitz (leveraged) 3 assets: ', 'Stocks', weights_ma_u_3[0].round(5),'Bonds', weights_ma_u_3[1].round(5), 'RF', weights_ma_u_3[2].round(5))
print('Expected return:', (exp_return_ma_u_3).round(4))
print('std:', (risk_ma_u_3).round(6))

Markowitz (leveraged) 3 assets:  Stocks 0.52362 Bonds 0.7354 RF -0.25902
Expected return: 0.0075
std: 0.02696


### Markowitz (leveraged - cvxpy optimization): 3 assets


In [15]:
mu_3_array = np.array(mu_3)
sigma_3_array = np.array(sigma_3)

In [16]:
n = mu_3_array.shape[0]
w = cp.Variable(n)
portfolio_return = mu_3_array.T @ w
portfolio_variance = cp.quad_form(w, sigma_3_array)
objective = cp.Minimize(portfolio_variance)

constraints = [cp.sum(w) == 1,        # Sum of weights must be 1
               w[0] >= 0,              # No short-selling for asset 1
               w[1] >= 0,              # No short-selling for asset 2
               w[2] >= -0.5,           # Asset 3 can be short-sold up to 50%
               portfolio_return >= 0.0075]
problem = cp.Problem(objective, constraints)
problem.solve()
optimal_weights = w.value

print("Optimal portfolio weights:", optimal_weights)

Optimal portfolio weights: [ 0.52361778  0.73540008 -0.25901786]


In [26]:
weights_ma_l = optimal_weights
print("Markowitz (leveraged) 3 assets:", "Stocks", weights_ma_l[0].round(6), "Bonds", weights_ma_l[1].round(6), "RF", weights_ma_l[2].round(6))
print("Expected return:", np.dot(weights_ma_l, mu_3_array).round(4))
print("std:", np.sqrt(np.dot(weights_ma_l, np.dot(sigma_3_array, weights_ma_l))).round(5))

Markowitz (leveraged) 3 assets: Stocks 0.523618 Bonds 0.7354 RF -0.259018
Expected return: 0.0075
std: 0.02696


### Old:

close solution using excess returns

In [27]:
data_excess = data_minus_rf - data['RF'].values.reshape(-1,1)
mu_excess = data_excess.mean()
sigma_excess = data_excess.cov()
RF = data['RF'].mean()

In [28]:
RF = data['RF'].mean()
mu_excess_target = 0.0075 - RF

In [29]:
sigma_inv_excess = np.linalg.inv(sigma_excess)
B = mu_excess.transpose() @ sigma_inv_excess @ mu_excess
C = sigma_inv_excess @ mu_excess

weights_ma_l_old = mu_excess_target * (C / B)

In [30]:
exp_return_ma_l = np.dot(weights_ma_l_old, mu_excess) + RF
exp_excess_return_ma_l = np.dot(weights_ma_l_old, mu_excess)
risk_ma_l = np.sqrt(weights_ma_l_old @ sigma_excess @ weights_ma_l_old)

print('Markowitz (leveraged):', weights_ma_l_old[0].round(3), 'Risk free', 'Bonds', weights_ma_l_old[1].round(3), 'Stocks', (1-weights_ma_l_old.sum()).round(3))
print('Expected return:', exp_return_ma_l.round(4))
print('std:', risk_ma_l.round(4))
print('Risk free mean return:', RF.round(4))

Markowitz (leveraged): 0.523 Risk free Bonds 0.739 Stocks -0.261
Expected return: 0.0075
std: 0.0269
Risk free mean return: 0.0021


### Risk parity:

In [31]:
def risk_parity_fun(w,sigma):
    std = np.sqrt(w @ sigma @ w)
    N  = len(w)
    w_rp = std**2 / ((sigma @ w) * N)
    objective_rp = np.sum((w - w_rp)**2)
    
    return objective_rp

In [39]:
constraints_rp = ({
    'type': 'ineq',
    'fun': lambda weights: 1.5 - np.sum(weights)  # This should sum weights to exactly 1
})


bounds_rp = [(0, None) for _ in range(len(initial_weights))]

initial_weights_rp = np.array([0.5, 0.5])
result_rp = minimize(risk_parity_fun, initial_weights_rp, args=(sigma_excess), method='SLSQP', bounds=bounds_rp, constraints=constraints_rp)

print("Risk Parity Portfolio Weights:", result_rp.x.round(3))
print("Expected excess return:", np.dot(result_rp.x, mu_excess).round(4))
print("std:", np.sqrt(risk_parity_fun(result_rp.x, sigma_excess)).round(4))
print("risk contribution per asset:", ((result_rp.x[0] * (sigma @ result_rp.x).iloc[0])/np.sqrt(risk_parity_fun(result_rp.x, sigma))).round(4), 
      ((result_rp.x[1] * (sigma @ result_rp.x).iloc[1])/np.sqrt(risk_parity_fun(result_rp.x, sigma))).round(4))

Risk Parity Portfolio Weights: [0.157 0.329]
Expected excess return: 0.0018
std: 0.0
risk contribution per asset: 0.0157 0.016


In [58]:
k = mu_excess_target / (mu_excess @ result_rp.x)
weights_rp = k * result_rp.x

exp_return_rp = np.dot(weights_rp, mu_excess) + RF
exp_excess_return_rp = np.dot(weights_rp, mu_excess)
risk_rp = np.sqrt(weights_rp @ sigma_excess @ weights_rp)

print('Risk Parity Portfolio Weights (leveraged):', weights_rp.round(3))
print('Expected return:', exp_return_rp.round(4))
print('std:', risk_rp.round(4))
print('Risk free weight:', (1-weights_rp.sum()).round(3))

Risk Parity Portfolio Weights (leveraged): [0.457 0.955]
Expected return: 0.0075
std: 0.0275
Risk free weight: -0.411


In [59]:
plot_data = pd.DataFrame({
    'Markowitz (unleveraged)': np.append(weights_ma_u[::-1], [0, exp_return_ma_u, risk_ma_u]),
    'Markowitz (leveraged)': np.append(weights_ma_l, [exp_return_ma_l, risk_ma_l]), 
    '60/40': np.append(weights_60_40[::-1], [0, exp_return_60_40, risk_60_40]),  
    'Risk Parity': np.append(weights_rp,[1-weights_rp.sum(), exp_return_rp, risk_rp])  
}, index=['Bonds', 'Stocks', 'RF', 'Expected Return', 'std'])  
plot_data = plot_data.round(4)

In [60]:
plot_data

,Markowitz (unleveraged),Markowitz (leveraged),60/40,Risk Parity
Bonds,0.3629,0.5236,0.4000,0.4568
Stocks,0.6371,0.7354,0.6000,0.9546
RF,0.0000,-0.2590,0.0000,-0.4115
Expected Return,0.0075,0.0075,0.0073,0.0075
std,0.0286,0.0269,0.0272,0.0275
